# Data Preprocessing - Harvesting Schedule
*This notebook contains the preprocessing steps performed to create a clean and structured dataset for further processing and analysis.
The provided data to be worked with is contained in several Excel sheets and one log-file, these will be imported and cleaned separately initially, then processed into a suitable format for further analysis.*

In [1]:
# imports
import pandas as pd
import os
import re
pd.options.mode.chained_assignment = None  # default='warn'

## 🏁 Goal
We need a tabular dataset that contains for the three selected harvesting schedules (MaxRes Solution, MaxWood solution, Stakeholder solution) the following columns:
- UG (stand ID)
- Period
- Presc (prescription ID)
- ShPer

We then need to create, for each of the three harvesting schedules, a matrix (FIR)st telling if UG number s during period t needs:
- 10m road access (1),
- no 10m road access (0),  

and a matrix (TIM)st telling if UG number s during period t needs:
- 5m road access (1),
- no 5m road access (0).

## 0.A. 📂 Prescription data sheet

In [2]:
# Load prescription data from excel sheet
file_path = r'0_Received_Data\3-Harvesting_Schedule\Presc_dataset21_missingstakeholders_sol.xlsx'

# Read 1st sheet into df
presc_raw = pd.read_excel(file_path, sheet_name=0)

# Look at DataFrame shape and columns
print('df shape:', presc_raw.shape)
print('columns:', presc_raw.columns)
presc_raw

df shape: (317883, 41)
columns: Index(['UG', 'ug_Sp', 'area', 'presc', 'Control', 'Na sol MaxRES',
       'Na sol MaxWood', 'period', 'species', 'c', 'v (m3)', 'vthin (m3) ',
       'vharv (m3)', 'vthin (ton)', 'vharv (ton)', 'Vremovido(ton)', 'fmm1',
       'fmm2', 'fmm3', 'fmm4', 'fmm5', 'fmm6', 'fmm7', 'fmm8', 'fmm9', 'fmm10',
       'fmm11', 'npv0', 'npv5', 'npv10', 'biodiversity0', 'biodiversity5',
       'biodiversity10', 'rait0', 'rait5', 'rait10', 'rit0', 'rit5', 'rit10',
       'erosion', 'Cortiça'],
      dtype='object')


,UG,ug_Sp,area,presc,Control,Na sol MaxRES,Na sol MaxWood,period,species,c,...,biodiversity5,biodiversity10,rait0,rait5,rait10,rit0,rit5,rit10,erosion,Cortiça
0,1,1_Ec,42.601782,1,1.1000,NaN,NaN,1,Ec,20290.7145,...,68.8904,73.4017,132.0655,166.1470,136.3257,0.9880,0.9963,0.9880,81862.7924,0.0000
1,1,1_Ec,42.601782,1,1.1000,NaN,NaN,2,Ec,36915.3071,...,68.6481,72.9810,140.5859,166.1470,140.5859,0.9912,0.9963,0.9883,78103.5629,0.0000
2,1,1_Ec,42.601782,1,1.1000,NaN,NaN,3,Ec,31452.4225,...,68.7367,73.1087,144.8461,166.1470,140.5859,0.9919,0.9967,0.9895,77765.0774,0.0000
3,1,1_Ec,42.601782,1,1.1000,NaN,NaN,4,Ec,33321.7817,...,67.4269,69.3376,161.8868,170.4071,161.8868,0.9959,0.9975,0.9959,70974.4382,0.0000
4,1,1_Ec,42.601782,1,1.1000,NaN,NaN,5,Ec,31034.9927,...,68.7400,73.1555,149.1062,166.1470,140.5859,0.9918,0.9966,0.9891,81249.0464,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317878,1626,1626_Pb,0.761332,503700,1626.5037,NaN,NaN,6,Sb,30.8645,...,1.0996,1.1132,3.0453,3.0453,3.0453,4.7000,4.8000,4.8000,633.4482,151.3299
317879,1626,1626_Pb,0.761332,503700,1626.5037,NaN,NaN,7,Sb,42.4194,...,1.1208,1.1306,3.0453,3.0453,3.0453,5.0000,5.0000,5.0000,19.9468,232.7392
317880,1626,1626_Pb,0.761332,503700,1626.5037,NaN,NaN,8,Sb,51.7995,...,1.1461,1.1556,3.0453,3.0453,3.0453,5.0000,5.0000,5.0000,12.5449,232.2747
317881,1626,1626_Pb,0.761332,503700,1626.5037,NaN,NaN,9,Sb,59.9478,...,1.1713,1.1808,3.0453,3.0453,3.0453,5.0000,5.0000,5.0000,12.5572,231.5058


In [141]:
pd.DataFrame(presc_raw['UG'].unique(), columns=['UG']).to_csv('1_Preprocessed_Data/0_Lists_of_Stands_Needing_Road_Access/all_UG.csv', index=False)

### 0. Subset & format the data
Create a smaller df:
- without the columns that are not to be used in this problem,
- that only contains data for the timeframe under study, i.e. the first 5 time periods (=50 years).

In [142]:
# Create the subset of columns to be used
presc = presc_raw[['UG', 'ug_Sp','species', 'presc', 'area', 'Control', 'Na sol MaxRES',
       'Na sol MaxWood', 'period', 'v (m3)', 'vthin (m3) ',
       'vharv (m3)', 'vthin (ton)', 'vharv (ton)', 'Vremovido(ton)', 'rait0', 'rait5', 'rait10']].copy()
print('shape after subsetting columns:', presc.shape)
presc.head(1)

shape after subsetting columns: (317883, 18)


,UG,ug_Sp,species,presc,area,Control,Na sol MaxRES,Na sol MaxWood,period,v (m3),vthin (m3),vharv (m3),vthin (ton),vharv (ton),Vremovido(ton),rait0,rait5,rait10
0,1,1_Ec,Ec,1,42.601782,1.1,NaN,NaN,1,1222.6712,0.0,4443.3659,0.0,2999.271982,2999.271982,132.0655,166.147,136.3257


In [143]:
# decide format, capitalize some columns, also strip the spaces from column names
presc = presc.astype({'UG': str, 
    'ug_Sp': str, 
    'species': str,
    'presc': str, 
    'Control': str, 
    'Na sol MaxRES': object, 
    'Na sol MaxWood': object, 
    'period': int,
    'v (m3)': float, 
    'vthin (m3) ': float, 
    'vharv (m3)': float, 
    'vthin (ton)': float, 
    'vharv (ton)': float, 
    'Vremovido(ton)': float, 
    'rait0': float, 
    'rait5': float, 
    'rait10': float})

# capitalize col name presc
presc.rename(columns={'presc': 'Presc'}, inplace=True)

# remove spaces from col names
presc.columns = presc.columns.str.replace(' ', '')

presc.head(1)

,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10
0,1,1_Ec,Ec,1,42.601782,1.1,NaN,NaN,1,1222.6712,0.0,4443.3659,0.0,2999.271982,2999.271982,132.0655,166.147,136.3257


In [144]:
# subset the first 5 time periods
print('before subsetting:', len(presc), 'rows')
presc['period'] = presc['period'].astype(int)
presc = presc[presc['period']<=5]
print('subset period 1-5:', len(presc), 'rows')

before subsetting: 317883 rows
subset period 1-5: 169693 rows


In [145]:
#count number of different stands
presc['UG'].nunique()

1406

### 1. 🔍 Check data consistency/plausibility

In [146]:
# check for duplicates
presc.duplicated(keep=False).sum()

0

In [147]:
# Get a list of columns with NaN values
columns_with_nan = presc.columns[presc.isna().any()].tolist()
print("Columns with NaN values:", columns_with_nan)

Columns with NaN values: ['NasolMaxRES', 'NasolMaxWood', 'vthin(m3)']


In [148]:
# look at the other values in the columns
print(presc['NasolMaxRES'].value_counts())
print(presc['NasolMaxWood'].value_counts())
print(presc['vthin(m3)'].value_counts())

NasolMaxRES
Yes    8212
Name: count, dtype: int64
NasolMaxWood
Yes    7670
Name: count, dtype: int64
vthin(m3)
0.0000      125557
192.0672        44
231.2539        44
363.6956        44
687.1360        44
             ...  
170.0766         1
717.6061         1
255.6229         1
318.6551         1
264.3240         1
Name: count, Length: 9617, dtype: int64


Check if columns

- `Control` == `UG`.`Presc`
- `ug_Sp` == `UG`_`species`
- (`vthin(ton)` = 0) IFF (`vthin(m3)` = 0)
- (`vharv(ton)` = 0) IFF (`vharv(m3)` = 0)
- `vthin(ton)`+`vharv(ton)` == `Vremovido(ton)`

In [149]:
## some data plausibility checks

print(len(presc), '`rows')
# Check if concatenating 'UG' with a dot ('.') and 'Presc' gives the 'Control' column
check_control_column = (presc['UG'] + '.' + presc['Presc']) == presc['Control']

if check_control_column.all(): 
    print("\u2713 column 'Control' is consistent. Can be dropped.")
else: print("\u26A0 column 'Control' has", len(presc) - check_control_column.sum(), 'inconsistencies')

# Check if concatenating 'UG' with species gives the 'ug_sp' column
check_ugsp_column = (presc['UG'] + '_' + presc['species']) == presc['ug_Sp']

if check_ugsp_column.all(): 
    print("\u2713 column 'ug_Sp' is consistent. Can be dropped.")
else: print("\u26A0 column 'ug_Sp' has", len(presc) - check_ugsp_column.sum(), 'inconsistencies')

# Check if vthin (ton) and vthin (m3) are either both zero or both nonzero
bmask_thinning = ((presc['vthin(ton)'] == 0) & (presc['vthin(m3)'] == 0)) | ((presc['vthin(ton)'] != 0) & (presc['vthin(m3)'] != 0))
if bmask_thinning.all():
    print("\u2713 Thinning")
else: 
    print("\u26A0 Thinning:")
print(f"There are {len(presc) - bmask_thinning.sum()} rows where 'vthin(ton)' and 'vthin(m3)' are inconsistent.")

# Check if vharv (ton) and vharv (m3) are either both zero or both nonzero
bmask_harvest = ((presc['vharv(ton)'] == 0) & (presc['vharv(m3)'] == 0)) | ((presc['vharv(ton)'] != 0) & (presc['vharv(m3)'] != 0))
if bmask_harvest.all():
    print("\u2713 Harvest:")
else:
    print(f"\u26A0 Harvest:")
print(f"There are {len(presc) - bmask_harvest.sum()} rows where 'vharv(ton)' and 'vharv(m3)' are inconsistent.")

# check if the amount of removed timber equals the sum of harvested and thinned timber
bmask_removido = presc['Vremovido(ton)'] == (presc['vthin(ton)'] + presc['vharv(ton)'])
if bmask_removido.all():
    print("\u2713 Removed wood:")
else:
    print(f"\u26A0 Harvest:")
print(f"There are {len(presc) - bmask_removido.sum()} rows where the sum of 'vthin (ton)' and 'vharv (ton)' does not equal 'Vremovido(ton)'.")

169693 `rows
⚠ column 'Control' has 45902 inconsistencies
⚠ column 'ug_Sp' has 108525 inconsistencies
⚠ Thinning:
There are 5565 rows where 'vthin(ton)' and 'vthin(m3)' are inconsistent.
✓ Harvest:
There are 0 rows where 'vharv(ton)' and 'vharv(m3)' are inconsistent.
✓ Removed wood:
There are 0 rows where the sum of 'vthin (ton)' and 'vharv (ton)' does not equal 'Vremovido(ton)'.


⚠️ We need to address the inconsistencies between the columns
- `vthin(ton)` and `vthin(m3)`,
- `ug_Sp` and `UG`_`species`,
- `Control` and `UG`.`Presc`

### 2. Address data inconsistencies

#### a. Resolve inconsistencies between `vthin(ton)` and `vthin(m3)` 

In [150]:
# show some rows where thinning values are inconsistent:
print('# of rows with inconsistent thinning values:', len(presc[~bmask_thinning]))
presc[~bmask_thinning].head(3)

# of rows with inconsistent thinning values: 5565


,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10
492,1,1_Ec,Ec,9001,42.6,1.9001,NaN,NaN,1,7263.30,NaN,0.0,0.0,0.0,0.0,140.58,166.14,136.32
493,1,1_Ec,Ec,9001,42.6,1.9001,NaN,NaN,2,12975.96,NaN,0.0,0.0,0.0,0.0,51.12,170.40,153.36
494,1,1_Ec,Ec,9001,42.6,1.9001,NaN,NaN,3,16916.46,NaN,0.0,0.0,0.0,0.0,42.60,178.92,157.62


Problem seems to be caused by NaN values in `vthin(m3)`. Replace them by 0 and check if the problem persists:

In [151]:
# replace NaN by 0
presc['vthin(m3)'].fillna(0, inplace=True)
presc[~bmask_thinning].head(2)

C:\Users\johan\AppData\Local\Temp\ipykernel_16600\2889342294.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  presc['vthin(m3)'].fillna(0, inplace=True)


,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10
492,1,1_Ec,Ec,9001,42.6,1.9001,NaN,NaN,1,7263.30,0.0,0.0,0.0,0.0,0.0,140.58,166.14,136.32
493,1,1_Ec,Ec,9001,42.6,1.9001,NaN,NaN,2,12975.96,0.0,0.0,0.0,0.0,0.0,51.12,170.40,153.36


In [152]:
# Validate: vthin(ton) and vthin(m3) are either both zero or both nonzero
bmask_thinning = ((presc['vthin(ton)'] == 0) & (presc['vthin(m3)'] == 0)) | ((presc['vthin(ton)'] != 0) & (presc['vthin(m3)'] != 0))
if bmask_thinning.all():
    print("\u2713 Thinning:")
else: 
    print("\u26A0 Thinning:")
print(f"There are {len(presc) - bmask_thinning.sum()} rows where 'vthin(ton)' and 'vthin(m3)' are inconsistent.")

✓ Thinning:
There are 0 rows where 'vthin(ton)' and 'vthin(m3)' are inconsistent.


#### b. Resolve inconsistencies between `ug_Sp` and `Ug`_`species` 

In [153]:
# look at rows where ug_Sp is not consistent with species or UG
print(len(presc[~check_ugsp_column]), 'rows where ug_Sp does not match UG _ species')
presc[~check_ugsp_column].head(1)

108525 rows where ug_Sp does not match UG _ species


,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10
30,1,1_Ec,Ct,1312,42.601782,1.1312,NaN,NaN,1,0.0,0.0,0.0,0.0,0.0,0.0,40.2712,51.1221,63.9027


`ug_Sp` seems to be only inconsistent with `species` column, not `UG`.
Validation that `ug_Sp` is consistent with `UG`:

In [154]:
# check if ug in UG column and ug in ug_sp column are the same
import re
# create column ug coming from ug_sp for comparison
presc['UG_from_ug_SP'] = presc['ug_Sp'].apply(lambda x: re.search(r'(\d+)_', x).group(1))

# check if ug in UG column and ug_sp are the same
(presc['UG'] == presc['UG_from_ug_SP']).all()

True

UG values are consistent, so the inconsistencies are only between `species` and `ug_Sp`.

In [155]:
# look at more data for one Prescription value from the rows where ug_Sp is not consistent with species
presc[presc['Presc']=='1312'].head(5)

,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10,UG_from_ug_SP
30,1,1_Ec,Ct,1312,42.601782,1.1312,NaN,NaN,1,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,40.2712,51.1221,63.9027,1
31,1,1_Ec,Ec,1312,42.601782,1.1312,NaN,NaN,1,0.0000,0.0000,4443.3659,0.000000,2999.271982,2999.271982,57.0695,110.7646,115.0248,1
32,1,1_Ec,Ct,1312,42.601782,1.1312,NaN,NaN,2,941.2790,0.0000,0.0000,0.000000,0.000000,0.000000,146.2009,170.4071,213.0089,1
33,1,1_Ec,Ct,1312,42.601782,1.1312,NaN,NaN,3,4879.1639,65.1807,0.0000,50.840946,0.000000,50.840946,151.4155,170.4071,213.0089,1
34,1,1_Ec,Ct,1312,42.601782,1.1312,NaN,NaN,4,6782.4034,2023.5847,0.0000,1578.396066,0.000000,1578.396066,160.0579,170.4071,213.0089,1


In this example, the `species` changes from Ec to Ct. during period 1. While `species` column indicates the species change, the column `ug_Sp` does not contain that information.

--> drop column `ug_Sp`.

#### c. Resolve inconsistencies between `Control` and `UG`.`Presc`

In [156]:
# look at rows where control is not consistent with UG or presc
bad_rows = presc[~check_control_column]
print(len(bad_rows), 'rows where control does not match UG Presc')
bad_rows.head(1)

45902 rows where control does not match UG Presc


,UG,ug_Sp,species,Presc,area,Control,NasolMaxRES,NasolMaxWood,period,v(m3),vthin(m3),vharv(m3),vthin(ton),vharv(ton),Vremovido(ton),rait0,rait5,rait10,UG_from_ug_SP
74,1,1_Ec,Ec,1500,42.601782,1.15,NaN,NaN,1,0.0,0.0,4443.3659,0.0,2999.271982,2999.271982,110.7646,115.0248,85.2036,1


`Control`(=UG.Presc) seems to be only inconsistent with `Presc`. Validate consistency between `Control`and `UG`:

In [157]:
# check if ug in UG column and Control column are the same
# create column ug coming from ug_sp for comparison
presc['UG_from_control'] = presc['Control'].apply(lambda x: re.search(r'(\d+).', x).group(1))

# check if ug in UG column and Control column are the same
(presc['UG'] == presc['UG_from_control']).all()

True

UG values in columns `UG` and `Control`(=UG.Presc) are consistent. Thus the inconsistencies are only  between `Presc` column and `Control` column.

In [158]:
# compare Presc column with presc value in control column in inconsistent cases:
# create column Presc coming from ug_sp for comparison
presc['Presc_from_control'] = presc['Control'].apply(lambda x: re.search(r'\.(.*)', x).group(1))

# look at the inconsistent cases
presc[presc['Presc_from_control']!=presc['Presc']][['Presc', 'Presc_from_control', 'Control']]

,Presc,Presc_from_control,Control
74,1500,15,1.15
75,1500,15,1.15
76,1500,15,1.15
77,1500,15,1.15
78,1500,15,1.15
...,...,...,...
317873,503700,5037,1626.5037
317874,503700,5037,1626.5037
317875,503700,5037,1626.5037
317876,503700,5037,1626.5037


Seems like column `Presc` has two additional zeros at the end. Validate this:

In [159]:
# check if all the inconsistent Presc values end with "00"
bad_rows['Presc'].str.endswith('00').all()

False

In [160]:
# look at inconsistent rows where the Presc value is Presc_from_control + "00"
bad_rows = presc[~check_control_column]
mask00 = bad_rows['Presc'] == bad_rows['Presc_from_control'] + '00' #str.endswith('00').sum(), 'inconsistent rows ending with 00')

# Apply the mask to find the rows where the condition is met
bad_rows_00 = bad_rows[mask00]

print(len(bad_rows_00), "inconsistent rows with Presc = Presc_from_control + '00'")
print(len(bad_rows), 'inconsistent rows in total')

42628 inconsistent rows with Presc = Presc_from_control + '00'
45902 inconsistent rows in total


Not all inconsistent rows' `Presc` values end with "00". Look at the remaining ones:

In [161]:
# show `Presc` values that are inconsistent with control but do not end with "00":
bad_rows[~bad_rows['Presc'].str.endswith('00')][['UG','Control','Presc']]

,UG,Control,Presc
512,2,2.2,20
513,2,2.2,20
514,2,2.2,20
515,2,2.2,20
516,2,2.2,20
...,...,...,...
317432,1623,1623.2,20
317433,1623,1623.2,20
317434,1623,1623.2,20
317435,1623,1623.2,20


Inconsistent rows' `Presc` values seem to have an additional 0 at the end. Validate this:

In [162]:
# check if all Presc values in inconsistent columns have an additional 0 at the end
bad_rows['Presc'].str.endswith('0').all()

True

In [163]:
# look at inconsistent rows where the Presc value is Presc_from_control + "0"
mask0 = bad_rows['Presc'] == bad_rows['Presc_from_control'] + '0' #str.endswith('00').sum(), 'inconsistent rows ending with 00')

# Apply the mask to find the rows where the condition is met
bad_rows_0 = bad_rows[mask0]

print(len(bad_rows_0), "inconsistent rows with Presc = Presc_from_control + '0'")
print(len(bad_rows_00), "inconsistent rows with Presc = Presc_from_control + '00'")
print(len(bad_rows), 'inconsistent rows in total')

# validate
checksum = bad_rows['Presc'].str.endswith('00').sum() + (bad_rows['Presc'].str.endswith('0') & ~bad_rows['Presc'].str.endswith('00')).sum()

if len(bad_rows)==checksum:
    print('\u2713 checksum', checksum)
else:
    print('\u26A0 checksum', checksum)

3274 inconsistent rows with Presc = Presc_from_control + '0'
42628 inconsistent rows with Presc = Presc_from_control + '00'
45902 inconsistent rows in total
✓ checksum 45902


We drop the `Control` column and keep the `Presc`, because in solution data also exists a column `Presc` and we need to be consistent there for merging later.

### 🗑️ Keep only the necessary columns

In [164]:
# Define the columns to keep
columns_to_keep = [
    'UG', 'species', 'Presc', 'area', 
    'NasolMaxRES', 'NasolMaxWood', 'period', 
    'Vremovido(ton)', 'rait0', 'rait5', 'rait10'
]

# Subset the DataFrame to include only the specified columns
presc = presc[columns_to_keep]
presc.head(1)

,UG,species,Presc,area,NasolMaxRES,NasolMaxWood,period,Vremovido(ton),rait0,rait5,rait10
0,1,Ec,1,42.601782,NaN,NaN,1,2999.271982,132.0655,166.147,136.3257


## 0.B. 📂📂Solution data sheets (Maxres / Maxwood)
The data we are looking at are three different solutions to a linear program (a previous work of Susete Marques). MaxRes and MaxWood solutions are coming from two separate excel sheets. The stakeholder solution was provieded in a different format and needs to be transformed to the same format as MaxRes and MaxWood.

####  load MaxWood and MaxRes solution

In [165]:
# Read solution sheets into dfs
sol_mres = pd.read_excel('0_Received_Data/3-Harvesting_Schedule/Presc_dataset21_missingstakeholders_sol.xlsx', sheet_name=3)
sol_mres = sol_mres.astype(str)
sol_mres.name = "MaxRes"

sol_mwood = pd.read_excel('0_Received_Data/3-Harvesting_Schedule/Presc_dataset21_missingstakeholders_sol.xlsx', sheet_name=2)
sol_mwood = sol_mwood.astype(str)
sol_mwood.name = "MaxWood"

for sol in [sol_mres, sol_mwood]:
    print(sol.name, sol.shape)
    print(sol.head(1))
    print('-----------------------------------------------------------------')

MaxRes (1512, 7)
  Decision Variable name  Control Presc UG  Sp ShPer  Sol
0      Presc1601_Pa1_Ec_5  1.1601  1601  1  Ec     5  Yes
-----------------------------------------------------------------
MaxWood (1512, 7)
  Decision Variable name  Control Presc UG  Sp ShPer  Sol
0      Presc3502_Pa1_Ec_0  1.3502  3502  1  Ec     0  Yes
-----------------------------------------------------------------


In [166]:
sol_mres.head

<bound method NDFrame.head of       Decision Variable name     Control   Presc    UG  Sp ShPer  Sol
0          Presc1601_Pa1_Ec_5     1.1601    1601     1  Ec     5  Yes
1         Presc19301_Pa2_Ec_0    2.19301   19301     2  Ec     0  Yes
2            Presc21_Pa3_Ec_5       3.21      21     3  Ec     5  Yes
3        Presc500308_Pa4_Pb_5   4.500308  500308     4  Pb     5  Yes
4           Presc503_Pa5_Pb_5      5.503     503     5  Pb     5  Yes
...                       ...        ...     ...   ...  ..   ...  ...
1507       Presc2_Pa1620_Ec_5     1620.2       2  1620  Ec     5  Yes
1508       Presc3_Pa1622_Ec_5     1622.3       3  1622  Ec     5  Yes
1509      Presc21_Pa1623_Ec_5    1623.21      21  1623  Ec     5  Yes
1510      Presc36_Pa1626_Ec_5    1626.36      36  1626  Ec     5  Yes
1511  Presc500700_Pa1626_Pb_5  1626.5007  500700  1626  Pb     5  Yes

[1512 rows x 7 columns]>

#### 🔍 check data consistency of MaxWood and MaxRes solution

In [167]:
# Check for duplicates and NaN
print("Duplicates in sol_mres:", sol_mres.duplicated(subset=['UG', 'Presc']).sum())

# Get a list of columns with NaN values in sol_mres
columns_with_nan = sol_mres.columns[sol_mres.isna().any()].tolist()
print("sol_mres columns with NaN values:", columns_with_nan)

print('--------------------')
print("Duplicates in sol_mwood:", sol_mwood.duplicated(subset=['UG', 'Presc']).sum())

# Get a list of columns with NaN values in sol_mwood
columns_with_nan = sol_mwood.columns[sol_mwood.isna().any()].tolist()
print("sol_mwood columns with NaN values:", columns_with_nan)

Duplicates in sol_mres: 0
sol_mres columns with NaN values: []
--------------------
Duplicates in sol_mwood: 0
sol_mwood columns with NaN values: []


Check if:
1. `Decision Variable name` == Presc`Presc`_ Pa`UG` _ `species`_ `ShPer`
2. `Control` == `UG`.`Presc`
3. `Sol` == 1 in every row

In [168]:
## Check data consistency of solmres and solmwood

solutions = [(sol_mres, 'sol_mres'), (sol_mwood, 'sol_mwood')]
for sol, name in solutions:
    print('-----------------------------------------------------------------')
    print(name)

    # change dtypes
    sol = sol.astype(str)
    
    # Check if concatenating Presc	UG	Sp	ShPer gives the Decision Variable name column
    check_Dvar_column = ('Presc'+ sol['Presc'] + "_Pa" + sol['UG'] + "_" + sol['Sp'] + "_" + sol['ShPer']) == sol['Decision Variable name ']

    if check_Dvar_column.all(): 
        print("\u2713 column 'Decision Variable name ' is consistent. Can be dropped.")
    else: print("\u26A0 column 'Decision Variable name ' has a problem.")

    # Remove trailing zeros 
    sol['Control'] = sol['Control'].str.rstrip('0')
    sol['Presc'] = sol['Presc'].str.rstrip('0')

    # Check if concatenating 'UG' with a dot ('.') and 'Presc' gives the 'Control' column
    check_control_column = (sol['UG'] + '.' + sol['Presc']) == sol['Control']

    if check_control_column.all(): 
        print("\u2713 column 'Control' is consistent. Can be dropped.")
    else: print("\u26A0 column 'Control' has a problem.")

    # Check if column 'Sol' contains values other than 'Yes'
    if len(sol['Sol'].unique()) != 1:
        # Display the other values
        print('\u26A0', sol['Sol'].unique())
    else:
        print("\u2713 column 'Sol' contains no additional information. Can be dropped.")

-----------------------------------------------------------------
sol_mres
✓ column 'Decision Variable name ' is consistent. Can be dropped.
✓ column 'Control' is consistent. Can be dropped.
✓ column 'Sol' contains no additional information. Can be dropped.
-----------------------------------------------------------------
sol_mwood
✓ column 'Decision Variable name ' is consistent. Can be dropped.
✓ column 'Control' is consistent. Can be dropped.
✓ column 'Sol' contains no additional information. Can be dropped.


In [169]:
# subset the dataframes 
solutions = [(sol_mres, 'sol_mres'), (sol_mwood, 'sol_mwood')]
for sol, name in solutions:
    print('-------------------')
    print(name)

   # change the order and drop unnecessary columns
    sol = sol[['UG', 'Presc', 'ShPer']]
    print(sol.shape)
    print(sol.head(1))

    # Update the original DataFrame
    if name == 'sol_mres':
        sol_mres = sol
    elif name == 'sol_mwood':
        sol_mwood = sol

-------------------
sol_mres
(1512, 3)
  UG Presc ShPer
0  1  1601     5
-------------------
sol_mwood
(1512, 3)
  UG Presc ShPer
0  1  3502     0


##### look at mixed stands

In [170]:
# number of stands in the solutions
mres_stands = sol_mres.drop_duplicates(subset='UG').reset_index(drop=True)
mres_stands

,UG,Presc,ShPer
0,1,1601,5
1,2,19301,0
2,3,21,5
3,4,500308,5
4,5,503,5
...,...,...,...
1401,1618,36700,5
1402,1620,2,5
1403,1622,3,5
1404,1623,21,5


In [171]:
mres_dupl = sol_mres[sol_mres.duplicated(subset='UG', keep=False)].sort_values(by='UG').reset_index(drop=True)
print(len(mres_dupl),'duplicate stands')
mres_dupl

212 duplicate stands


,UG,Presc,ShPer
0,1015,503,5
1,1015,3,5
2,1037,35700,0
3,1037,700,0
4,1070,500700,5
...,...,...,...
207,977,501700,5
208,978,21700,5
209,978,500700,5
210,979,500700,5


There are 212 stands that have more than one prescription in MaxRes.

In [172]:
mres_dupl = sol_mres[sol_mres.duplicated(subset='UG', keep='last')].sort_values(by='UG').reset_index(drop=True)
len(mres_dupl)

106

There are 106 stands that have 2 prescriptions in MaxRes.

In [173]:
mwood_dupl = sol_mwood[sol_mwood.duplicated(subset='UG', keep=False)].sort_values(by='UG').reset_index(drop=True)
print(len(mwood_dupl))
mwood_dupl = sol_mwood[sol_mwood.duplicated(subset='UG', keep='last')].sort_values(by='UG').reset_index(drop=True)
len(mwood_dupl)

212


106

In [174]:
mwood_stands = sol_mwood.drop_duplicates(subset='UG').reset_index(drop=True)
len(mwood_stands)

1406

Same for MWood. 1406 stands, 106 of them with two prescriptions.

## 0.C 📂 Solution log file (stakeholder solution)

### 1. Bring stakeholder solution to tabular format


We need the columns `UG`, `Presc` and `ShPer`:

In [175]:
# Extract stakeholder solution from logfile

log_file_path = '0_Received_Data/3-Harvesting_Schedule/Stakeholders_Solution.log'

# Open and read the log file
with open(log_file_path, 'r') as file:
    log_content = file.read()

# Use regular expression to find the variables and their values
pattern = r'(Presc\S+)\s+(\d+\.\d{6})'
matches = re.findall(pattern, log_content)

# Extract variables and their respective values
variables_values = re.findall(r'(Presc\S+)\s+(\d+\.\d{6})', log_content)

# Create a DataFrame from the matches
sol_stake = pd.DataFrame(matches, columns=['Variable Name', 'Solution Value'])

# Convert the 'Solution Value' column to float
sol_stake['Solution Value'] = sol_stake['Solution Value'].astype(float)

print(sol_stake.shape)
sol_stake.head(1)

(1514, 2)


,Variable Name,Solution Value
0,Presc503_Pa101_Pb_10,1.0


In [176]:
# Extract the required parts of the variable name into seperate columns via regex
sol_stake['UG'] = sol_stake['Variable Name'].apply(lambda x: re.search(r'_Pa(\d+)_', x).group(1))
#sol_stake['ug_Sp'] = sol_stake['Variable Name'].apply(lambda x: re.search(r'_Pa(\d+_[A-Za-z]{2})_', x).group(1))
sol_stake['Presc'] = sol_stake['Variable Name'].apply(lambda x: re.search(r'Presc(\d+)_Pa', x).group(1))
sol_stake['ShPer'] = sol_stake['Variable Name'].apply(lambda x: re.search(r'_[A-Za-z]{2}_(\d+)$', x).group(1))

sol_stake.head(1)

,Variable Name,Solution Value,UG,Presc,ShPer
0,Presc503_Pa101_Pb_10,1.0,101,503,10


In [177]:
# validate that all extracted UG, Presc and ShPer are contained in Variable Name
count = sol_stake.apply(lambda row: f"Pa{row['UG']}" in row['Variable Name'], axis=1).sum()
print("Number of rows where 'Pa<UG>' is contained in 'Variable Name':", count)
count = sol_stake.apply(lambda row: f"Presc{row['Presc']}" in row['Variable Name'], axis=1).sum()
print("Number of rows where 'Presc' is contained in 'Variable Name':", count)
count = sol_stake.apply(lambda row: f"_{row['ShPer']}" in row['Variable Name'], axis=1).sum()
print("Number of rows where '_Shper' is contained in 'Variable Name':", count)

Number of rows where 'Pa<UG>' is contained in 'Variable Name': 1514
Number of rows where 'Presc' is contained in 'Variable Name': 1514
Number of rows where '_Shper' is contained in 'Variable Name': 1514


In [178]:
# drop column Variable name
sol_stake = sol_stake[['UG','Presc','ShPer', 'Solution Value']]
sol_stake.head(1)

,UG,Presc,ShPer,Solution Value
0,101,503,10,1.0


### 2. Check data consistency of stakeholder solution

In [179]:
# check for duplicates and NaN
print("Duplicates in sol_stake:")
print(sol_stake.duplicated(subset=['UG', 'Presc']).sum())

# Get a list of columns with NaN values in sol_mres
columns_with_nan = sol_stake.columns[sol_stake.isna().any()].tolist()
print("Columns with NaN values:", columns_with_nan)

Duplicates in sol_stake:
0
Columns with NaN values: []


In [180]:
# Check if column 'Solution Value' contains values other than 1
if len(sol_stake['Solution Value'].unique()) > 1:
    # Display the other values
    print('\u26A0 there are different solution values:', sol_stake['Solution Value'].unique())
else:
    print("\u2713 column 'Solution Value' contains no additional information. Can be dropped.")

⚠ there are different solution values: [1.       0.494295 0.567668 0.505705 0.432332]


Expecting only binary `Solution Value`s ("Is the prescription *x* for UG *y* selected?", yes/no), we surprisingly find some **fractional solution values**:

In [181]:
# look at solution values that are not 1:
weird = sol_stake[sol_stake['Solution Value']!=1].sort_values(by='UG')
weird

,UG,Presc,ShPer,Solution Value
298,1253,36503,10,0.494295
1134,1253,36,5,0.505705
776,1339,19700,0,0.567668
1155,1339,19318,0,0.432332


Observations:
- There are two `UG`s (1253 and 1339), with two different prescriptions `Presc` each, where the `Solution Value` is <1.
- For each `UG`, the two fractional `Solution Value`s add up to 1.

In [182]:
# Group by 'UG' and sum 'Solution Value'
partial_sums = weird.groupby('UG')['Solution Value'].sum().reset_index()

print("\nPartial Sums per UG:")
partial_sums


Partial Sums per UG:


,UG,Solution Value
0,1253,1.0
1,1339,1.0


Considering the context (*Use prescription "x" for UG "y": YES/NO*), all solution values are supposed to be binary. The present fractional solution values indicate that a **linear relaxation** was used when obtaining the stakeholder solution. 
The fractional values are most likely resulting from the linear relaxation of the problem: Supposedly, not only 0 or 1, but any value within a given range (0,1), were allowed as solution values.

### 3. Correcting Non-Binary Solution Values in stakeholder solution
To interpret solution values from a linearly relaxed binary problem, common approaches include the rounding method and the threshold method. We apply these methods by replacing values above 0.5 with 1 and values below 0.5 with 0, and thereby obtain the following:

In [183]:
# correct the solution values
threshold = 0.5
weird['Corrected Value'] = (weird['Solution Value'] >= threshold).astype(int)
print(weird)

        UG  Presc ShPer  Solution Value  Corrected Value
298   1253  36503    10        0.494295                0
1134  1253     36     5        0.505705                1
776   1339  19700     0        0.567668                1
1155  1339  19318     0        0.432332                0


In [184]:
# Update original dataframe with corrected binary values
sol_stake.loc[sol_stake['Solution Value'] < 1, 'Solution Value'] = weird['Corrected Value']
print('sol_stake updated with corrected values (binary)')

sol_stake updated with corrected values (binary)


In [185]:
# validate
if len(sol_stake['Solution Value'].unique()) > 1:
    # Display the other values
    print('\u26A0 there are different solution values:', sol_stake['Solution Value'].unique())
else:
    print("\u2713 column 'Solution Value' contains no additional information. Can be dropped.")

⚠ there are different solution values: [1. 0.]


In [186]:
# drop the rows that were not selected for the solution
sol_stake = sol_stake[sol_stake['Solution Value'] == 1]
print('shape after drop:', sol_stake.shape)

shape after drop: (1512, 4)


In [187]:
# validate that now there are only solution values ==1
if len(sol_stake['Solution Value'].unique()) > 1:
    # Display the other values
    print('\u26A0 there are different solution values:', sol_stake['Solution Value'].unique())
else:
    print("\u2713 column 'Solution Value' contains no additional information. Can be dropped.")

✓ column 'Solution Value' contains no additional information. Can be dropped.


In [188]:
sol_stake = sol_stake[['UG','Presc','ShPer']]
sol_stake.head()

,UG,Presc,ShPer
0,101,503,10
1,1015,503,5
2,1016,500602,0
3,102,503,5
4,103,503,5


In [189]:
stake_stands = sol_stake.drop_duplicates(subset='UG').reset_index(drop=True)
len(stake_stands)

1406

In [190]:
stake_dupl = sol_stake[sol_stake.duplicated(subset='UG', keep=False)].sort_values(by='UG').reset_index(drop=True)
print(len(stake_dupl))
stake_dupl = sol_stake[sol_stake.duplicated(subset='UG', keep='last')].sort_values(by='UG').reset_index(drop=True)
len(stake_dupl)

212


106

## 1. 🔗 Merge prescription data and solution data

To obtain the information in the required format for further work, we need to merge each solution with the prescription data.

- Data needed from the prescription sheet: UG, Presc, species, area, period, Vremovido, rait0/5/10
- Data needed from the solutions sheet: UG, Presc, ShPer


### 0. Prep for merge

In [191]:
# Formatting all dfs as string
sol_mres = sol_mres.astype(str)
sol_mwood = sol_mwood.astype(str)
sol_stake = sol_stake.astype(str)
presc = presc.astype(str)

In [192]:
# Check for duplicates (again)
print("Duplicates in sol_mres:")
print(sol_mres.duplicated(subset=['UG', 'Presc']).sum())

print("Duplicates in sol_mwood:")
print(sol_mwood.duplicated(subset=['UG', 'Presc']).sum())

print("Duplicates in sol_stake:")
print(sol_stake.duplicated(subset=['UG', 'Presc']).sum())

Duplicates in sol_mres:
0
Duplicates in sol_mwood:
0
Duplicates in sol_stake:
0


In [193]:
# Check for duplicates (again)
print("Duplicates UG in sol_mres:")
print(sol_mres.duplicated(subset=['UG']).sum())

print("Duplicates UG in sol_mwood:")
print(sol_mwood.duplicated(subset=['UG']).sum())

print("Duplicates UG in sol_stake:")
print(sol_stake.duplicated(subset=['UG']).sum())

Duplicates UG in sol_mres:
106
Duplicates UG in sol_mwood:
106
Duplicates UG in sol_stake:
106


In [194]:
sol_mres[sol_mres.duplicated(subset=['UG'])==True].sort_values(by='UG')

,UG,Presc,ShPer
977,1015,503,5
999,1037,35700,0
1033,1070,500700,5
1037,1073,500700,5
1057,1093,503,5
...,...,...,...
933,975,500700,5
935,976,500700,5
937,977,501700,5
939,978,500700,5


### a. MaxWood & MaxRes solution

#### 1. Filter prescription df for only rows (prescriptions) selected for MaxWood or MaxRES solution

In [195]:
# look at the possible values that describe if a prescription is part of maxwood /maxres solution
print(presc['NasolMaxWood'].unique())
print(presc['NasolMaxRES'].unique())

['nan' 'Yes']
['nan' 'Yes']


In [196]:
# filter prescription data for prescriptions contained in mres and mwood solution 
filter_mres = presc[presc['NasolMaxRES'] == 'Yes']
print('length mres:', len(filter_mres))

filter_mwood = presc[presc['NasolMaxWood'] == 'Yes']
print('length mwood:', len(filter_mwood))

length mres: 8212
length mwood: 7670


In [197]:
# subset for the needed columns
filter_mres = filter_mres[['UG', 'Presc', 'species', 'area', 'period', 'Vremovido(ton)', 'rait0', 'rait5', 'rait10']]
filter_mwood = filter_mwood[['UG', 'Presc', 'species',  'area', 'period', 'Vremovido(ton)', 'rait0', 'rait5', 'rait10']]
print('remaining columns:')
filter_mres.head(1)

remaining columns:


,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10
129,1,1601,Ec,42.60178231,1,2999.2719825,110.7646,115.0248,85.2036


#### 2. Merge to obtain `ShPer` from solutions sheet

In [198]:
# left merge mres with data from solution sheet to get shper values
print('mres before merge shape:', filter_mres.shape, sol_mres.shape)
mres = pd.merge(filter_mres, sol_mres, how='outer', on=['UG', 'Presc'])
print('after merge shape:', mres.shape)

# left merge mwood with data from solution sheet to get shper values
print('mwood before merge shape:', filter_mwood.shape, sol_mwood.shape)
mwood = pd.merge(filter_mwood, sol_mwood, how='outer', on=['UG', 'Presc'])
print('after merge shape:', mwood.shape)


mres before merge shape: (8212, 9) (1512, 3)
after merge shape: (8212, 10)
mwood before merge shape: (7670, 9) (1512, 3)
after merge shape: (7670, 10)


In [199]:
mres.head(1)

,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
0,1,1601,Ec,42.60178231,1,2999.2719825,110.7646,115.0248,85.2036,5


#### 3. Validate the merged data
There should be no NaN values in mres or mwood:

In [200]:
# check for NaN values in mres and mwood

# Check for NaN values in mres
print("NaN values in mres:")
print(mres.isna().sum().sum())

# Check for NaN values in mwood
print("NaN values in mwood:")
print(mwood.isna().sum().sum())

NaN values in mres:
0
NaN values in mwood:
0


We should get the same mres (mwood) dfs by directly merging prescription data (df) and sol_mres (sol_mwood), then filter out the NaNs values after. Validate this:

In [201]:
# validating the obtained merged dfs for maxres and maxwood

print('MaxRes')
print('before merge shapes:', presc.shape, sol_mres.shape)
mres2 = pd.merge(presc, sol_mres, how='left', on=['UG', 'Presc'])
print('after merge shape:', mres2.shape)
mres2 = mres2[mres2['ShPer'].notna()]
print('after filtering out NaN values shape:', mres2.shape)

print('------------------------')

print('maxWood')
print('before merge shape:', presc.shape, sol_mwood.shape)
mwood2 = pd.merge(presc, sol_mwood, how='left', on=['UG', 'Presc'])
print('after merge shape:', mwood2.shape)
mwood2 = mwood2[mwood2['ShPer'].notna()]
print('after filtering out NaN shape:', mwood2.shape)

MaxRes
before merge shapes: (169693, 11) (1512, 3)
after merge shape: (169693, 12)
after filtering out NaN values shape: (8212, 12)
------------------------
maxWood
before merge shape: (169693, 11) (1512, 3)
after merge shape: (169693, 12)
after filtering out NaN shape: (7670, 12)


In [202]:
mres.head(1)

,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
0,1,1601,Ec,42.60178231,1,2999.2719825,110.7646,115.0248,85.2036,5


In [203]:
mwood.head(1)

,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
0,1,3502,Ec,42.60178231,1,3968.3560500000003,136.3257,149.1062,119.285,0


### b. Stakeholder solution

Merge to obtain `ShPer` from solution data

In [204]:
# merge with data from solution data to get shper value
print('before merge shape:', presc.shape, sol_stake.shape)
stake = pd.merge(presc, sol_stake, how='outer', on=['UG', 'Presc'])
print('after merge shape:', stake.shape)
stake = stake[stake['ShPer'].notna()]
print('after filtering out NaN values shape:', stake.shape)

before merge shape: (169693, 11) (1512, 3)
after merge shape: (169693, 12)
after filtering out NaN values shape: (8391, 12)


In [205]:
stake.head(2)

,UG,species,Presc,area,NasolMaxRES,NasolMaxWood,period,Vremovido(ton),rait0,rait5,rait10,ShPer
59,1,Ec,1601,42.60178231,Yes,nan,1,2999.2719825,110.7646,115.0248,85.2036,5
60,1,Qr,1601,42.60178231,Yes,nan,1,0.0,51.1221,63.9027,51.1221,5


### 4. Validation

In [206]:
# check if all solution  data (stake, mres, mwood) contain the same number of unique UGs
unique_ug_mwood = mwood['UG'].nunique()
unique_ug_mres = mres['UG'].nunique()
unique_ug_stake = stake['UG'].nunique()

# Print the number of unique UG values
print(f"Unique UG values in mwood: {unique_ug_mwood}")
print(f"Unique UG values in mres: {unique_ug_mres}")
print(f"Unique UG values in stake: {unique_ug_stake}")

Unique UG values in mwood: 1406
Unique UG values in mres: 1406
Unique UG values in stake: 1406


In [207]:
# check for NaN values
print(f"NaN values in mwood: {mwood.isna().sum().sum()}\n")
print(f"NaN values in mres: {mres.isna().sum().sum()}\n")
print(f"NaN values in stake: {stake.isna().sum().sum()}\n")

NaN values in mwood: 0

NaN values in mres: 0

NaN values in stake: 0



## 🔍 Look at a special cases:
### a. Different species planted together in the same stand
In the provided harvesting schedules (=solutions), we have, for some `UG`, 
- in the same `period`, 
- two different prescriptions `Presc`,
- (for two different  `species`).

These are "**mixed stands**" where two different species are prescribed/planted together:

In [208]:
# filter mixed stands in mres and look at some example rows
mres_dupl = mres[mres.duplicated(subset=['UG','period'], keep=False)].copy()
mres_mixed = mres_dupl[~mres_dupl.duplicated(subset=['UG','period', 'Presc'], keep=False)].sort_values(by=['UG','period'])
print(len(mres_dupl))
print(len(mres_dupl[mres_dupl.duplicated(subset=['UG','period', 'Presc'], keep=False)]))
print(len(mres_mixed))
mres_mixed.head(4)

2221
1304
917


,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
104,1015,3,Ec,2.934230183,1,47.336467500000005,9.683,11.7369,11.1501,5
109,1015,503,Pb,8.036243378,1,0.0,8.0362,9.6435,8.8399,5
105,1015,3,Ec,2.934230183,2,102.99150000000002,9.683,11.7369,11.4435,5
110,1015,503,Pb,8.036243378,2,0.0,9.6435,11.2507,10.4471,5


### b. Species changes

In the harvesting schedules, we have for some `UG` two rows of data
- in the same `period`
- with the same prescription `presc`
- but with two different `species`.

These are species changes, as already observed when looking at the full prescription data.

In [209]:
# species changes in mres
mres_change = mres[mres.duplicated(subset=['UG','period', 'Presc'], keep=False)].sort_values(by=['UG','period'])
print('# of species changes in MaxRes:', len(mres_change)/2)
mres_change.head(4)

# of species changes in MaxRes: 652.0


,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
0,1,1601,Ec,42.60178231,1,2999.2719825,110.7646,115.0248,85.2036,5
1,1,1601,Qr,42.60178231,1,0.0,51.1221,63.9027,51.1221,5
7,10,21700,Ec,15.59066931,2,3141.32499,3.1181,15.5907,15.5907,5
8,10,21700,Sb,15.59066931,2,0.0,62.3627,62.3627,62.3627,5


In [210]:
# species changes in mwood
mwood_change = mwood[mwood.duplicated(subset=['UG','period', 'Presc'], keep=False)].sort_values(by=['UG','period'])
print('# of species changes in MaxWood:', len(mwood_change)/2)
mwood_change.head(4)

# of species changes in MaxWood: 110.0


,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer
0,1,3502,Ec,42.60178231,1,3968.3560500000003,136.3257,149.1062,119.285,0
1,1,3502,Pb,42.60178231,1,0.0,4.2602,4.2602,4.2602,0
226,1038,23502,Ec,16.20559165,1,835.3823625,8.1028,8.1028,8.1028,5
227,1038,23502,Pb,16.20559165,1,0.0,14.585,16.2056,14.585,5


In [211]:
# species changes in stakeholder sol
stake_change = stake[stake.duplicated(subset=['UG','period', 'Presc'], keep=False)].sort_values(by=['UG','period'])
print('# of species changes in Stakeholder solution:', len(stake_change)/2)
stake_change.head(4)

# of species changes in Stakeholder solution: 831.0


,UG,species,Presc,area,NasolMaxRES,NasolMaxWood,period,Vremovido(ton),rait0,rait5,rait10,ShPer
59,1,Ec,1601,42.60178231,Yes,nan,1,2999.2719825,110.7646,115.0248,85.2036,5
60,1,Qr,1601,42.60178231,Yes,nan,1,0.0,51.1221,63.9027,51.1221,5
488,1000,Ct,19319,13.47439105,nan,nan,1,0.0,21.3602,33.686,33.686,10
489,1000,Ec,19319,13.47439105,nan,nan,1,2032.7803200000003,12.3929,33.686,33.686,10


## 3. Select the correct fire resistance value (rait) based on shrub cleaning periodicity
Due to varying shrub cleaning periodicities (0, 5, or 10), corresponding fire resistance values are available in the `rait0`, `rait5`, and `rait10` columns.


### Step 1: Select the fire resistance value based on shrub cleaning periodicity.

In [212]:
# Select the correct fire resistance value depending on shrub cleaning periodicity

# Define a function to select the correct 'rait' based on 'ShPer' value
def fill_rait(row):
    if row['ShPer'] == 0:
        return row['rait0']
    elif row['ShPer'] == 5:
        return row['rait5']
    elif row['ShPer'] == 10:
        return row['rait10']
    else:
        return None  # or any default value you prefer for other cases

# Use the function defined above in a loop to select the correct rait
for name, presc in [("mres", mres), ("mwood", mwood), ("stake", stake)]:
    print(name)
    # Ensure correct format
    presc['ShPer'] = presc['ShPer'].astype(int)

    # Apply the function to create the 'Rait' column
    presc['Rait'] = presc.apply(fill_rait, axis=1)

    # Look at the result
    print(presc.head(1))
    print('---------------------------------------------------')

mres
  UG Presc species         area period Vremovido(ton)     rait0     rait5  \
0  1  1601      Ec  42.60178231      1   2999.2719825  110.7646  115.0248   

    rait10  ShPer      Rait  
0  85.2036      5  115.0248  
---------------------------------------------------
mwood
  UG Presc species         area period      Vremovido(ton)     rait0  \
0  1  3502      Ec  42.60178231      1  3968.3560500000003  136.3257   

      rait5   rait10  ShPer      Rait  
0  149.1062  119.285      0  136.3257  
---------------------------------------------------
stake
   UG species Presc         area NasolMaxRES NasolMaxWood period  \
59  1      Ec  1601  42.60178231         Yes          nan      1   

   Vremovido(ton)     rait0     rait5   rait10  ShPer      Rait  
59   2999.2719825  110.7646  115.0248  85.2036      5  115.0248  
---------------------------------------------------


### Step 2: Validate if all rait values are correctly selected

In [213]:
# check if the Rait was assigned correctly

for i in [mres, mwood, stake]:

    # Check if value in 'Rait' column matches 'rait0' when 'ShPer' is 0
    is_rait0 = (i['Rait'] == i['rait0']) & (i['ShPer'] == 0)
    print(is_rait0.sum(), "rows have Rait == rait0 based on ShPer==0")

    # Check if value in 'Rait' column matches 'rait5' when 'ShPer' is 5
    is_rait5 = (i['Rait'] == i['rait5']) & (i['ShPer'] == 5)
    print(is_rait5.sum(), "rows have Rait == rait5 based on ShPer==5")

    # Check if value in 'Rait' column matches 'rait10' when 'ShPer' is 10
    is_rait10 = (i['Rait'] == i['rait10']) & (i['ShPer'] == 10)
    print(is_rait10.sum(), "rows have Rait == rait10 based on ShPer==10")

    if (is_rait10.sum()+is_rait5.sum()+is_rait0.sum())==len(i):
        print('\u2713', (is_rait10.sum()+is_rait5.sum()+is_rait0.sum()), 'out of', len(i),'were matched correctly')
    else:
        print('u26A0', (is_rait10.sum()+is_rait5.sum()+is_rait0.sum()), 'out of', len(i),'were matched correctly')
    print('--------------------------------------------------')

535 rows have Rait == rait0 based on ShPer==0
7365 rows have Rait == rait5 based on ShPer==5
312 rows have Rait == rait10 based on ShPer==10
✓ 8212 out of 8212 were matched correctly
--------------------------------------------------
3323 rows have Rait == rait0 based on ShPer==0
1926 rows have Rait == rait5 based on ShPer==5
2421 rows have Rait == rait10 based on ShPer==10
✓ 7670 out of 7670 were matched correctly
--------------------------------------------------
4295 rows have Rait == rait0 based on ShPer==0
3440 rows have Rait == rait5 based on ShPer==5
656 rows have Rait == rait10 based on ShPer==10
✓ 8391 out of 8391 were matched correctly
--------------------------------------------------


In [214]:
mres.head()

,UG,Presc,species,area,period,Vremovido(ton),rait0,rait5,rait10,ShPer,Rait
0,1,1601,Ec,42.60178231,1,2999.2719825,110.7646,115.0248,85.2036,5,115.0248
1,1,1601,Qr,42.60178231,1,0.0,51.1221,63.9027,51.1221,5,63.9027
2,1,1601,Qr,42.60178231,2,0.0,170.4071,213.0089,170.4071,5,213.0089
3,1,1601,Qr,42.60178231,3,2.8856880000000005,170.4071,213.0089,170.4071,5,213.0089
4,1,1601,Qr,42.60178231,4,11.048232,170.4071,213.0089,170.4071,5,213.0089


## 4. 💾 Save cleaned data to csv files

In [215]:
# subset the data, only keep the necessary
columns_to_keep = [
    'UG', 'species', 'Presc', 'period', 'Vremovido(ton)', 'Rait'
]

# Subset each DataFrame to include only the columns to keep
mres = mres[columns_to_keep]
mwood = mwood[columns_to_keep]
stake = stake[columns_to_keep]

In [216]:
print('mres shape:', mres.shape)
print('mwood shape:', mwood.shape)
print('stake shape:', stake.shape)
stake.head(1)

mres shape: (8212, 6)
mwood shape: (7670, 6)
stake shape: (8391, 6)


,UG,species,Presc,period,Vremovido(ton),Rait
59,1,Ec,1601,1,2999.2719825,115.0248


In [217]:
# Save the selected data to CSV files

# Define subfolders
subfolder = '1_Preprocessed_Data'
subsubfolder = os.path.join(subfolder, '0_Prescription_Data')

# Create subfolders if they do not exist
os.makedirs(subsubfolder, exist_ok=True)

for data, filename in [(mres,'MaxRes_Data.csv'), (mwood,'MaxWood_Data.csv'), (stake,'Stakeholder_Data.csv')]:
    file_path = os.path.join(subsubfolder, filename)
    data.to_csv(file_path, index=False)
    print(' ✓ saved', filename, 'to csv file, path', file_path)

 ✓ saved MaxRes_Data.csv to csv file, path 1_Preprocessed_Data\0_Prescription_Data\MaxRes_Data.csv
 ✓ saved MaxWood_Data.csv to csv file, path 1_Preprocessed_Data\0_Prescription_Data\MaxWood_Data.csv
 ✓ saved Stakeholder_Data.csv to csv file, path 1_Preprocessed_Data\0_Prescription_Data\Stakeholder_Data.csv


## 5. Create road access matrices

We wanna create, for each of the three harvesting schedules, a firetruck road access matrix telling if a specific `UG` number during a specific `period` needs:
- 10m road access (1),
- no 10m road access (0),  

and a timber road access matrix telling if `UG` during `period` needs:
- 5m road access (1),
- no 5m road access (0).

To validate we also create for each solution, and each period (1 to 5):
- a list of UG that need timbertruck road access (5m)
- a list of UG that need firetruck road access (10m) 

### 0. Extract the stands + validation

In [218]:
# Extract unique UG values from each DataFrame and compare for validation
ug_mres = sol_mres.drop_duplicates(subset=['UG'])['UG'].astype(int).sort_values().astype(str).reset_index(drop=True)
ug_mwood = sol_mwood.drop_duplicates(subset=['UG'])['UG'].astype(int).sort_values().astype(str).reset_index(drop=True)
ug_stake = sol_stake.drop_duplicates(subset=['UG'])['UG'].astype(int).sort_values().astype(str).reset_index(drop=True)
ug_df = presc.drop_duplicates(subset=['UG'])['UG'].astype(int).sort_values().astype(str).reset_index(drop=True)

# Check if all unique UG values are the same
if ug_mres.equals(ug_mwood) and ug_mres.equals(ug_stake) and ug_mres.equals(ug_df):
    print('All matching.')
else:
    print("Matching error: The unique UG values do not match across the DataFrames.")


All matching.


In [219]:
stands = ug_df.to_frame()  # or use ug_mwood or ug_stake, they are the same

In [220]:
stands.head()

,UG
0,1
1,2
2,3
3,4
4,5


### 1. Remove accessible stands from problem

In [221]:
folder_path = '1_Preprocessed_Data/1_Stand_Accessibility'

# Load the CSV files into DataFrames
stands_accessible = pd.read_csv(f'{folder_path}/stands_accessible.csv')
stands_timberroadonly = pd.read_csv(f'{folder_path}/stands_timberroadonly.csv')
stands_inaccessible = pd.read_csv(f'{folder_path}/stands_inaccessible.csv')

# Rename the column to 'UG'
stands_inaccessible.columns = ['UG']
stands_timberroadonly.columns = ['UG']
stands_accessible.columns = ['UG']

print('fully accessible:', len(stands_accessible))
print('timber access:', len(stands_timberroadonly))
print('not accessible:', len(stands_inaccessible))
stands_inaccessible.head()

fully accessible: 324
timber access: 151
not accessible: 212


,UG
0,840
1,841
2,846
3,1204
4,1206


In [222]:
319+139+229

687

In [223]:
# Convert 'UG' to string in both DataFrames to avoid any data type mismatches
stands['UG'] = stands['UG'].astype(str)

In [224]:
# Data validation
#  Check if entries in 'stands' are contained in 'UG_ID' of 'accessible_stands'
stands_accessible['UG'] = stands_accessible['UG'].astype(str)
common_entries = stands[stands['UG'].isin(stands_accessible.UG)]

# Display the common entries (optional)
print(len(common_entries))

stands_timberroadonly['UG'] = stands_timberroadonly['UG'].astype(str)
common_entries = stands[stands['UG'].isin(stands_timberroadonly['UG'])]

# Display the common entries (optional)
print(len(common_entries))

stands_inaccessible['UG'] = stands_inaccessible['UG'].astype(str)
common_entries = stands[stands['UG'].isin(stands_inaccessible['UG'])]

# Display the common entries (optional)
print(len(common_entries))

324
151
212


### 2. create road access matrices

We are choosing that the stands within the lowest 0.05 quantile of rait value need fire road access.

#### Attention, special case
We can not simply keep any row from the duplicate ones, due to the species changes/mixed stands we need to make sure:

- When creating the timber road access matrix, in case of species changes, we need to keep the row with the higher V_removido value. -> simple, just filter for Vremovido >0
- when creating the bigroad access matrix, we keep the one with the lower rait value and drop the one with higher rait value. 

### 3. Create matrices

In [ ]:
# Initialize process info list
process_info_list = []

# Ensure 'stands_inaccessible' is a DataFrame and rename UG to ID_UG
stands_inaccessible = pd.DataFrame({'ID_UG': stands_inaccessible['UG']})

# Define the dataframes and their names
dataframes = [("mres", mres), ("mwood", mwood), ("stake", stake)]

# Create base matrix for stands_inaccessible
basematrix = stands_inaccessible.copy()
basematrix[[f'5mt{i}' for i in range(1, 6)]] = 0
basematrix[[f'10mt{i}' for i in range(1, 6)]] = 0

# Iterate through each DataFrame
for name, df in dataframes:
    print(f"Processing {name} - Initial shape:", df.shape)

    # Convert columns to appropriate types
    df['Vremovido(ton)'] = df['Vremovido(ton)'].astype(float)
    df['period'] = df['period'].astype(int)
    df['Rait'] = df['Rait'].astype(float)

    for period in range(1, 6):
        # Filter data for the current period
        period_df = df[df['period'] == period]

        # Identify stands needing timber access (Vremovido > 0) - only for stands_inaccessible
        stands_with_timber = period_df.loc[period_df['Vremovido(ton)'] > 0, 'UG']
        stands_with_timber = stands_with_timber[stands_with_timber.isin(stands_inaccessible['ID_UG'])]

        # Identify stands needing firetruck access (Rait in bottom 5%) - only for stands_inaccessible
        quantile_threshold = period_df['Rait'].quantile(0.05)
        stands_with_low_rait = period_df.loc[period_df['Rait'] < quantile_threshold, 'UG']
        stands_with_low_rait = stands_with_low_rait[stands_with_low_rait.isin(stands_inaccessible['ID_UG'])]

        # Update the matrix
        basematrix.loc[basematrix['ID_UG'].isin(stands_with_timber), f'5mt{period}'] = 1
        basematrix.loc[basematrix['ID_UG'].isin(stands_with_low_rait), f'10mt{period}'] = 1

    # Save final matrix
    output_folder = os.path.join('1_Preprocessed_Data', '0_Access_Requirements_Matrices')
    os.makedirs(output_folder, exist_ok=True)
    output_file = os.path.join(output_folder, f'{name}_stands_access_needs.csv')
    basematrix.to_csv(output_file, index=False)

print(f"Final access matrix saved to: {output_file}")


Processing mres - Initial shape: (8212, 6)
Processing mwood - Initial shape: (7670, 6)
Processing stake - Initial shape: (8391, 6)
Final access matrix saved to: 1_Preprocessed_Data\0_Access_Requirements_Matrices\stake_stands_access_matrix.csv


# create the matrices

process_info_list = []

# Ensure 'stands_inaccessible' and 'stands_timberroadonly' are DataFrames
stands_inaccessible = pd.DataFrame({'UG': stands_inaccessible['UG']})  # Assuming 'UG' is the column name in stands_inaccessible
stands_timberroadonly = pd.DataFrame({'UG': stands_timberroadonly['UG']})  # Same for stands_timberroadonly

# Define the dataframes and their names
dataframes = [("mres", mres), ("mwood", mwood), ("stake", stake)]

# Iterate through the dictionary and process each DataFrame
for name, df in dataframes:

    # Print info of initial df
    print(f"Initial shape of {name}:", df.shape)

    # Convert columns to appropriate types
    df['Vremovido(ton)'] = df['Vremovido(ton)'].astype(float)
    df['period'] = df['period'].astype(int)
    df['Rait'] = df['Rait'].astype(float)

    # Initialize matrices by assigning the first column the list of stands
    timberaccess = pd.DataFrame({'UG': stands_inaccessible.UG})
    fireaccess = pd.DataFrame({'UG': stands_inaccessible.UG.append(stands_timberroadonly.UG, ignore_index=True)})


    for period in range(1, 6):
        # Filter data for the current period
        period_df = df[df['period'] == period]
        # Filter only the stands that have timber
        stands_with_timber = period_df[period_df['Vremovido(ton)'] > 0]
        # filter the lowest rait stands (0.05 quant)
        quant = 0.05
        stands_with_low_rait = period_df[period_df['Rait'] < period_df['Rait'].quantile(quant)]

        # Subset columns to be used for validation
        stands_with_timber = stands_with_timber[['UG', 'Vremovido(ton)']]
        stands_with_low_rait = stands_with_low_rait[['UG', 'Rait']]

        # Update matrices
        timberaccess[f'need_5m_access_period{period}'] = timberaccess['UG'].isin(stands_with_timber['UG']).astype(int)
        fireaccess[f'need_10m_access_period{period}'] = fireaccess['UG'].isin(stands_with_low_rait['UG']).astype(int)

        # Define file paths
        subfolder = os.path.join('1_Preprocessed_Data', '0_Lists_of_Stands_Needing_Road_Access')
        os.makedirs(subfolder, exist_ok=True)
        tfile_path = os.path.join(subfolder, f'{name}_period{period}_timber.csv')
        ffile_path = os.path.join(subfolder, f'{name}_period{period}_fire.csv')

        # Initialize saved file variables
        saved_tfile = 'No data to save'
        saved_ffile = 'No data to save'

        # Save files if data is not empty
        if not stands_with_timber.empty:
            stands_with_timber.to_csv(tfile_path, index=False)
            saved_tfile = tfile_path

        if not stands_with_low_rait.empty:
            stands_with_low_rait.to_csv(ffile_path, index=False)
            saved_ffile = ffile_path

        # Append process info
        process_info_list.append({
            'DataFrame': name,
            'Period': period,
            'Filtered_Shape': stands_with_timber.shape,
            'Saved_Files': f'{saved_tfile} , {saved_ffile}'
        })

    # Save matrices
    matrixfolder = os.path.join('1_Preprocessed_Data', '0_Access_Requirements_Matrices')
    os.makedirs(matrixfolder, exist_ok=True)
    timberaccess.to_csv(os.path.join(matrixfolder, f'{name}_stands_needing_timber_access.csv'), index=False)
    fireaccess.to_csv(os.path.join(matrixfolder, f'{name}_stands_needing_firetruck_access.csv'), index=False)

In [226]:
process_info_list

[]

#### 4. Validate the creation of timber/fire access matrices

In [227]:
# define a function to check the creation of matrices (compare with created lists)
def check_matrices(matrix_folder, dataframes, periods, stands):
    for name, _ in dataframes:
        timber_file = os.path.join(matrix_folder, f'{name}_stands_needing_timber_access.csv')
        fire_file = os.path.join(matrix_folder, f'{name}_stands_needing_firetruck_access.csv')

        # Check if files exist
        if not os.path.isfile(timber_file):
            print(f"Error: Timber access matrix file {timber_file} does not exist.")
            continue
        if not os.path.isfile(fire_file):
            print(f"Error: Firetruck access matrix file {fire_file} does not exist.")
            continue

        # Load matrices
        timberaccess = pd.read_csv(timber_file)
        fireaccess = pd.read_csv(fire_file)

        # Check matrix dimensions
        expected_rows = len(stands)
        expected_cols = len(periods) + 1  # One column for 'UG' plus one for each period

        # Check timber access matrix
        for matrix, matrix_type in zip([timberaccess, fireaccess], ['timber', 'fire']):
            matrix_file = timber_file if matrix_type == 'timber' else fire_file

            # Check rows
            if matrix.shape[0] != expected_rows:
                print(f"Error: {matrix_type.capitalize()} matrix {matrix_file} has {matrix.shape[0]} rows instead of {expected_rows}.")

            # Check columns
            if matrix.shape[1] != expected_cols:
                print(f"Error: {matrix_type.capitalize()} matrix {matrix_file} has {matrix.shape[1]} columns instead of {expected_cols}.")

            # Check values
            if not matrix.iloc[:, 1:].isin([0, 1]).all().all():
                print(f"Error: {matrix_type.capitalize()} matrix {matrix_file} contains non-binary values.")

            print(f"{matrix_type.capitalize()} matrix {matrix_file} is correct.")

# Define paths and parameters
matrixfolder = os.path.join('1_Preprocessed_Data', 'Matrices_Road_Access')
dataframes = [("mres", mres), ("mwood", mwood), ("stake", stake)]
periods = range(1, 6)  # Periods 1 to 5
stands = pd.concat([df['UG'] for _, df in dataframes]).unique()  # Assuming 'UG' column contains stand identifiers

# Perform checks
check_matrices(matrixfolder, dataframes, periods, stands)

Error: Timber access matrix file 1_Preprocessed_Data\Matrices_Road_Access\mres_stands_needing_timber_access.csv does not exist.
Error: Timber access matrix file 1_Preprocessed_Data\Matrices_Road_Access\mwood_stands_needing_timber_access.csv does not exist.
Error: Timber access matrix file 1_Preprocessed_Data\Matrices_Road_Access\stake_stands_needing_timber_access.csv does not exist.
